### Fetching transcripts

In [348]:
from langchain_community.document_loaders.youtube import YoutubeLoader

In [364]:
url = "https://www.youtube.com/watch?v=KL6CjNxkZDQ"
loader = YoutubeLoader.from_youtube_url(url)
result = loader.load()
result[:4]

[Document(page_content="hi this is yugini uh so when it comes to dealing with asynchronous Frameworks if you never dealt with them before it might be confusing and the documentation often doesn't help dealing with this confusion because there are way too many concurrency related terms a plus overall it's relatively new thing not something that's been a while since forever so and not everyone uses it uh so sometimes it can be tricky to navigate this space altogether and so one of the approaches that follows because of that is just writing async in front of every function and just hoping for the best the other thing some people do is just writing up synchronously like they used to and ignoring this part completely uh I was also asked in my previous video why I didn't use async functions for tutorial and that's a good question uh and there are actually a couple reasons for why I didn't want to do it uh first I didn't want to make the video more complicated but today I want to talk about t

### SPlitting the transcript to smaller chunks for countering token limitation

In [350]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [351]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(result)
len(docs)

26

### Generating Embeddings and storing them in a faiss index

In [352]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS

In [353]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key="api")

In [354]:
db = FAISS.from_documents(docs, embeddings)
db.save_local("faiss_index")

### Loading Faiss index from local and chat with the bot

In [355]:
db = FAISS.load_local("faiss_index",embeddings=embeddings,allow_dangerous_deserialization=True)

In [356]:
from langchain import PromptTemplate

In [357]:
prompt_template = """
  Explore the YouTube video transcript for detailed insights. 
  If the answer is present in the context, provide comprehensive details. 
  If not, derive the answer from the video content anyhow.
  \n\n
  Transcript:\n {context}?\n
  Question: \n{question}\n

  Answer:
"""

prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])

In [358]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
model = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.3,google_api_key="api")

In [359]:
chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

In [360]:
question = "When to use async def and def"

In [361]:
docs = db.similarity_search(question,k=15)
docs

[Document(page_content="it returned at the same time so you might be thinking like well it seems that asynchronous keyword works you see yeah it's returned deployment done and all other stuff so it worked fine and didn't block anything but now let's see the regular read file synchronous that will use our synchronous function without a sync keyword as well and execute it and now let's try to grid and it also returns right away so I'll clear and show it also returns right away and here it also doing the same thing so then does it actually make any difference if we wrote uh asynchronous read file or synchronous one because the the this one just uses the regular one so what's the deal here so before we explore what actually is going on let's look at the other functions like this one so this file will this one will read the file and then use some calculations our CPU bound function that I showed you before but it will be written with the async keyword now uh read file and calculate synchron

In [363]:
response = chain(
    {"input_documents":docs, "question": question}
    , return_only_outputs=True)
response

{'output_text': '**Use `async def` when:**\n\n* Using asynchronous libraries that support the `async` keyword.\n* Gathering tasks to run concurrently using `async io.gather`.\n* Running computations in a separate process to avoid blocking the main thread.\n\n**Use `def` when:**\n\n* Using libraries that do not support the `async` keyword.\n* Reading file data without needing to return the result immediately.\n* Performing small computations that do not block the main thread.\n* Using regular databases that do not support asynchronous operations.'}

### Improving the responses

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
model = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.3,google_api_key="api",convert_system_message_to_human=True)
qa_chain = RetrievalQA.from_chain_type(
    model,
    retriever=db.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

In [ ]:
result = qa_chain({"query":question})
result["result"]

'The patterns discussed in the video are:\n- State\n- Mediator\n- Builder\n- Factory'